# Software-in-the-Loop

NOTE: Software-in-the-Loop interfaces as well as documentation are in alpha stage.

Vessim supports the integration of real applications running on hardware or virtualized environments.

In this example, we introduce a simple node that imitates workload and periodically communicates its power consumption to the `ComputingSystem` actor. 

For this, we can utilize Vessim's `SilController`:


In [13]:
from __future__ import annotations

from threading import Thread
from fastapi import FastAPI
import time
import requests

from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment, Microgrid
from vessim.power_meter import PowerMeter
from vessim.signal import Signal, HistoricalSignal
from vessim.sil import SilController, Broker, get_latest_event
from vessim.storage import SimpleBattery

# Jupyter async bug fix
import nest_asyncio

nest_asyncio.apply()

For this example, we want to give the Controller an API route to change the
minimum state of charge of our battery through HTTP. This works by defining the
`api_routes()` function which provides three arguments, we can use for our API
routes, namely the FastAPI application, the Vessim Broker and Grid Signals from
the Signal example.

In [14]:
def api_routes(
    app: FastAPI,
    broker: Broker,
    grid_signals: dict[str, Signal],
):
    @app.put("/battery/min-soc")
    async def put_battery_min_soc(min_soc: float):
        broker.set_event("battery_min_soc", min_soc)

We use the FastAPI `app` to define the HTTP endpoint. You can read more on the
use of FastAPI [here](https://fastapi.tiangolo.com/tutorial/). Behind the scenes of the Vessim SiL Controller is a key-value database that holds shared memory for the simulation and the API server process. The Vessim Broker conveys between the DB and the user. To save a value in this DB, you can set an event with a key, value pair. In this case: `"battery_min_soc"` and `min_soc`.

However, it is possible that multiple event between simulation steps occur. In
this case, you need to tell Vessim how you would like it to behave. We can do
this through the use of collectors. For this scenario, we simply want the most
recent value to be recognized:

In [ ]:
def battery_min_soc_collector(events: dict, microgrid: Microgrid, **kwargs):
    print(f"Received battery.min_soc events: {events}")
    assert isinstance(microgrid.storage, SimpleBattery)
    microgrid.storage.min_soc = get_latest_event(events)

Because we now have a real node that we want to feed its power consumption to
the Computing System, we need some mechanism to collect this consumption.
Instead of the usual mock power meter, we need to implement an HTTP power meter
that periodically poll the power consumption from the independent node. We
expect for now, that our remote node offers an HTTP endpoint that tells us its
power consumption.

In [ ]:
class HttpPowerMeter(PowerMeter):
    def __init__(
        self,
        name: str,
        port: int = 8000,
        address: str = "127.0.0.1",
        collect_interval: float = 1,
    ) -> None:
        super().__init__(name)
        self.port = port
        self.address = address
        self.collect_interval = collect_interval
        self._p = 0.0
        Thread(target=self._collect_loop, daemon=True).start()

    def measure(self) -> float:
        return self._p

    def _collect_loop(self) -> None:
        while True:
            self._p = float(
                requests.get(
                    f"{self.address}:{self.port}/power",
                ).text
            )
            time.sleep(self.collect_interval)

This HTTP power meter is subclassed from the Vessim `PowerMeter` class that is
accepted by the Computing System Actor.

Now we can connect all our components.

In [ ]:
environment = Environment(sim_start="15-06-2022")

monitor = Monitor()  # stores simulation result on each step
sil_controller = SilController(  # executes software-in-the-loop controller
    api_routes=api_routes,
    request_collectors={"battery_min_soc": battery_min_soc_collector},
)
environment.add_microgrid(
    actors=[
        ComputingSystem(power_meters=[HttpPowerMeter(name="sample_app", port=8001)]),
        Generator(signal=HistoricalSignal.from_dataset("solcast2022_global"), column="Berlin"),
    ],
    storage=SimpleBattery(capacity=100),
    controllers=[monitor, sil_controller],
    step_size=60,  # global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600, rt_factor=1, print_progress=False)
monitor.to_csv("result_csv")

Our independent node could e.g. look like this:
```python
from fastapi import FastAPI
from threading import Thread
import random
import time
import uvicorn


class NodeApiServer:
    def __init__(self, port: int, p_static: float, p_max: float):
        self.app = FastAPI()
        self.port = port
        self.p_static = p_static
        self.p_max = p_max
        self.utilization = 0
        Thread(target=self._workload_sim, daemon=True).start()

        @self.app.get("/power")
        async def get_power():
            return self.p_static + self.utilization * (self.p_max - self.p_static)

    def _workload_sim(self):
        while True:
            self.utilization = round(random.uniform(0.1, 1), 2)
            time.sleep(2)

    def start(self):
        uvicorn.run(self.app, host="0.0.0.0", port=self.port)


if __name__ == "__main__":
    NodeApiServer(port=8001, p_static=4, p_max=8).start()
```